In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
raw_data_path = Path("../data/raw/CPFL-RS-Informações industrial grupo A.xlsx").resolve()

processed_data_folder = Path("../data/processed").resolve()
processed_data_folder.mkdir(parents=True, exist_ok=True)

data = pd.read_excel(raw_data_path)
data.head(30)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consumo (KWH),Consumo (KWH),Consumo (KWH),...,FER (KWh),FER (KWh),FER (KWh),FER (KWh),FER (KWh),FER (KWh),FER (KWh),FER (KWh),FER (KWh),FER (KWh)
2,NaN,NaN,NaN,NaN,NaN,NaN,Ano/Mes Lançamento,01.2024,02.2024,03.2024,...,03.2024,04.2024,05.2024,06.2024,07.2024,08.2024,09.2024,10.2024,11.2024,12.2024
3,Empresa,Nº instalação,Classe de cálculo,SubGrupo de Tensão,Município,Item Faturamento BW,SubAgrupador de Fatu,KWH,KWH,KWH,...,KWH,KWH,KWH,KWH,KWH,KWH,KWH,KWH,KWH,KWH
4,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Cons Ponta Band Verde - TE,Consumo,463.296,543.6,564.432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Consumo F Ponta Band Verde-TE,Consumo,1365.264,1403.424,1835.376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Consumo Reativo Exc Fora Ponta,FER,NaN,NaN,NaN,...,386.102,413.523,422.763,394.793,352.517,303.499,38.893,643.854,344.034,363.77
7,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Consumo Reativo Exc Ponta,FER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Demanda [kW] - TUSD,Demanda,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Energ Atv Inj. mUC mPT - FP TE,Energia Compensada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
columns_vars = data.iloc[3,0:7].values
columns_months = data.iloc[2,7:19].values
data_columns = np.append(columns_vars, columns_months)

data_consumption = data.iloc[:,0:19]
data_consumption.columns = data_columns
data_consumption = data_consumption.drop([0,1,2,3])
data_consumption = data_consumption.melt(columns_vars).dropna(subset="value")
data_consumption["value"] = pd.to_numeric(data_consumption["value"])

data_demand = pd.concat([data.iloc[:,0:7], data.iloc[:,19:31]], axis=1)
data_demand.columns = data_columns
data_demand = data_demand.drop([0,1,2,3])
data_demand = data_demand.melt(columns_vars).dropna(subset="value")
data_demand["value"] = pd.to_numeric(data_demand["value"])

data_fer = pd.concat([data.iloc[:,0:7], data.iloc[:,31:]], axis=1)
data_fer.columns = data_columns
data_fer = data_fer.drop([0,1,2,3])
data_fer = data_fer.melt(columns_vars).dropna(subset="value")
data_fer["value"] = pd.to_numeric(data_fer["value"])

data_all = pd.concat(
    [data_consumption, data_demand, data_fer],
    axis=0
)

data_all.rename(columns={"variable": "Meses", "value": "Valores"}, inplace=True)
data_all

,Empresa,Nº instalação,Classe de cálculo,SubGrupo de Tensão,Município,Item Faturamento BW,SubAgrupador de Fatu,Meses,Valores
0,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Cons Ponta Band Verde - TE,Consumo,01.2024,463.296
1,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Consumo F Ponta Band Verde-TE,Consumo,01.2024,1365.264
7,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Energ Atv Inj. mUC oPT- Pta-TE,Energia Compensada,01.2024,-420.816
8,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Energia Atv Injetada Fponta TE,Energia Compensada,01.2024,-1365.264
9,Grupo RGE,Cliente 1,Industrial,A4,CAXIAS DO SUL,Energia Atv Injetada Ponta TE,Energia Compensada,01.2024,-42.480
...,...,...,...,...,...,...,...,...,...
30910,Grupo RGE,Cliente 347,Industrial,A4,CRUZ ALTA,USD Consumo Reativo Ponta,FER,12.2024,11413.662
30919,Grupo RGE,Cliente 348,Industrial,A4,IPE,USD Consumo Reativo Fora Ponta,FER,12.2024,5.771
30920,Grupo RGE,Cliente 348,Industrial,A4,IPE,USD Consumo Reativo Ponta,FER,12.2024,0.267
30924,Grupo RGE,Cliente 349,Industrial,A4,PORTAO,USD Consumo Reativo Fora Ponta,FER,12.2024,77.091


In [4]:
file_save_path = processed_data_folder / raw_data_path.name 
data_all.to_csv(file_save_path, index=False)